<a href="https://colab.research.google.com/github/Dwinurcahyo2/Data-Mining-Lanjut/blob/main/UAS_data_mining_lanjut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UAS DATA MINING LANJUT**

LIBRARY